<a href="https://colab.research.google.com/github/PrathibhaNishadi/Cricket-Squad-Selector/blob/main/T20Ball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.optimizers import Adam
import keras
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import locale
import os
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np
import glob
import cv2
import os
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model


def load_data_attributes(inputPath):
	df = pd.read_csv(inputPath)
	return df

def process_data_attributes(df, train, test):
  continuous = ["Tmatches","Twickets", "InEcon", "InSR"]
  trainX = (train[continuous])
  testX = (test[continuous])
  return (trainX, testX)

def create_mlp(dim, regress=False):
	model = Sequential()
	model.add(Dense(13, input_dim=4, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))

	if regress:
		model.add(Dense(1, activation="linear"))

	return model


print("[INFO] loading Cricket attributes...")

df = load_data_attributes("drive/My Drive/Project/TTBallTrain.csv")

print("[INFO] constructing training/testing split...")
(train, test) = train_test_split(df, test_size=0.10, random_state=42)
_trainY = train["Wickets"]
_testY = test["Wickets"]
trainY=[]

for i in _trainY:
  trainY.append(i)
print(trainY)
testY=[]

for i in _testY:
  testY.append(i)
print(testY)

trainY=np.array(trainY)
testY=np.array(testY)

print("[INFO] processing data...")
(trainX, testX) = process_data_attributes(df, train, test)

model = create_mlp(trainX.shape[1], regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss='mean_squared_error', optimizer='adam')


print("[INFO] training model...")

model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=200, batch_size=8)

model.save('TTBall_model.mod')


In [ ]:
import csv
import keras
import pandas as pd

predict=pd.read_csv("drive/My Drive/Project/TTBalling.csv")
labels = ["Matches","Wickets", "InEcon", "InSR"]
predictList=(predict[labels])

model=keras.models.load_model('TTBall_model.mod')
results=model.predict(predictList)
results=results



output = results.tolist()
names = (predict["Name"])
types = (predict["Type"])

myFile = open('TTBallResults.csv', 'w')  
with myFile:  
    myFields = ['Name','Type', 'Wickets']
    writer = csv.DictWriter(myFile, fieldnames=myFields)    
    writer.writeheader()
    
    for i in range(len(output)):
      writer.writerow({'Name' : names[i],'Type' : types[i], 'Wickets': output[i][0]})
      
      
# making data frame from csv file 
data = pd.read_csv("TTBallResults.csv")

# creating a rank column and passing the returned rank series 
data["Rank"] = data["Wickets"].rank(ascending=0)

# sorting w.r.t name column 
data.sort_values("Rank", inplace = True)  

export_csv = data.to_csv ('TTBallResults.csv', index = None, header=True)

data


,Name,Type,Wickets,Rank
3,Lasith Malinga,Bowler,1.092602,1.0
4,Akila Dananjaya,Allrounder,1.004107,2.0
5,Lakshan Sandakan,Bowler,0.968887,3.0
0,Suranga Lakmal,Bowler,0.943708,4.0
8,Jeffrey Vandersay,Bowler,0.929589,5.0
7,Isuru Udana,Allrounder,0.821314,6.0
2,Kasun Rajitha,Bowler,0.683335,7.0
9,Thisara Perera,Allrounder,0.634940,8.0
1,Dhananjaya de Silva,Allrounder,0.577351,9.0
6,Amila Aponso,Bowler,0.389113,10.0
